<h1>Emotion Sentiment Analysis</h1>

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("datasets/master.csv", encoding='utf-8')
df['content']= df['content'].astype(str)
emotion = df.emotion.unique()
df = df.reindex(np.random.permutation(df.index))
df.to_csv("datasets/master.csv", index = False)

In [3]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
vocabulary_size = 200000
tokenizer = Tokenizer(num_words= 1000)
tokenizer.fit_on_texts(df['content'])
sequences = tokenizer.texts_to_sequences(df['content'])
data = pad_sequences(sequences, maxlen=50)
X = data

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [13]:
import pandas as pd
import numpy as np
flag = pd.Series(list(df['emotion']))
Y = pd.get_dummies(flag)
Y = np.array(Y)

<h1>Using word embeddings to create a LSTM classification model</h1>

In [14]:
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical

In [15]:
X = X.reshape(len(X), 50)
inputs = Input(shape=[50])
layer = Embedding(100000,500,input_length=50)(inputs)
layer = LSTM(1000)(layer)
layer = Dense(256)(layer)
layer = Activation('relu')(layer)
layer = Dropout(0.5)(layer)
layer = Dense(9)(layer)
layer = Activation('softmax')(layer)
model = Model(inputs=inputs,outputs=layer)
model.summary()
model.compile(loss='categorical_crossentropy',optimizer="adam",metrics=['accuracy'])
model.fit(X,Y,batch_size=256,epochs=2,
          validation_split=0.6, shuffle=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 50)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 50, 500)           50000000  
_________________________________________________________________
lstm_1 (LSTM)                (None, 1000)              6004000   
_________________________________________________________________
dense_1 (Dense)              (None, 256)               256256    
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 9)                 2313      
__________

In [16]:
model.save("master_model.h5")

<h1>Prediction</h1>

In [4]:
from keras.models import load_model
model = load_model('master_model.h5')
statement = "How is your country"
sequences = tokenizer.texts_to_sequences(statement)
data = pad_sequences(sequences, maxlen=50)
y = model.predict(data)

In [15]:
argument = np.argmax(y[18], axis=0)
print(emotion[argument])

fear
